In [ ]:
import pandas as pd
import numpy as np
import math
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
print(os.listdir("/content/drive/My Drive"))

['Getting started.pdf', 'Lec1.pptx', 'BOY DEV CHAP 01.pdf', 'CHAPTER-TWO.pptx', 'BOY DEV CHAP 03.pdf', 'BOY DEV CHAP 02.pdf', 'Chapter 1-4 RE.docx', 'Chapter 5 & 6.docx', 'BOY DEV CHAP 04.pdf', 'Chapter 7 & 8.docx', 'Copy of AdminLTE.Template.sln', 'Assighnment-2.gdoc', 'Assighnment-3.gdoc', 'Bereket_Hailu_Results.pdf', 'ADDIS ABABA SCIENCE AND TECHNOLOGY UNIVERSITY.gdoc', 'Bereket Hailu_Resume.pdf', '{Ebook PDF Epub {Download} when-anger-hurts-quieting-the-storm-within-by-matthew-mckay.pdf', '{Ebook PDF Epub {Download} when-anger-hurts-quieting-the-storm-within-by-matthew-mckay.gdoc', '[PDF!] Download The Richest Man in Babylon- Original 1926 Edition - Clason, George S. Free Epub.pdf', '[PDF!] Download The Richest Man in Babylon- Original 1926 Edition - Clason, George S. Free Epub.gdoc', 'Tomasulo.pdf', 'Tomasulo.docx', 'Tomasulo.gdoc', 'Bereket Hailu-CV (2).pdf', 'Bereket Hailu-CV (1).pdf', 'Zabulon_Hailu_report-1(New).docx', 'Colab Notebooks', 'Crirtical Review_Bereket Hailu_GSR 272

# Data Preparation

## Load DGA From Netlab 360

In [ ]:
columns_dga = ["domain_name","start_time", "end_time","dga_family"]
Netlab_dga = pd.read_csv('/content/drive/My Drive/Colab Notebooks/netlab360.csv', names=columns_dga)
Netlab_dga.drop(["start_time", "end_time"], axis=1, inplace=True)
Netlab_dga.head()

,domain_name,dga_family
0,odqzesdvd.info,nymaim
1,mcrwnxdzu.info,nymaim
2,poxrscvfoy.biz,nymaim
3,dupzwi.info,nymaim
4,pgldibr.com,nymaim


In [ ]:
Netlab_dga.shape

(1048576, 2)

In [ ]:
columns_dga = ["domain_name","start_time", "end_time","dga_family"]
Netlab_dga2 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/netlab360_2.csv', names=columns_dga)
Netlab_dga2.drop(["start_time", "end_time"], axis=1, inplace=True)
Netlab_dga2.head()

,domain_name,dga_family
0,btausagabardinedazyx.com,banjori
1,omgysagabardinedazyx.com,banjori
2,fpdcsagabardinedazyx.com,banjori
3,aqvzsagabardinedazyx.com,banjori
4,skfasagabardinedazyx.com,banjori


In [ ]:
Netlab_dga2.shape

(440019, 2)

In [ ]:
# Concatenating both datasets
Netlab_dga_data = pd.concat([Netlab_dga, Netlab_dga2])

print("Whole Dataset has {} rows".format(Netlab_dga_data.shape[0]))
Netlab_dga_data.head()

Whole Dataset has 1488595 rows


,domain_name,dga_family
0,odqzesdvd.info,nymaim
1,mcrwnxdzu.info,nymaim
2,poxrscvfoy.biz,nymaim
3,dupzwi.info,nymaim
4,pgldibr.com,nymaim


In [ ]:
Netlab_dga_data['dga_family'].nunique()

56

In [ ]:
grouped=Netlab_dga_data['domain_name'].groupby(Netlab_dga_data['dga_family'])

In [ ]:
grouped.count()

dga_family
abcbot                27
antavmu               16
bamital              104
banjori           483072
bigviktor           1000
blackhole              2
ccleaner               1
chinad              1000
conficker            492
copperstealer         11
cryptolocker        1000
dircrypt             766
dyre                1000
emotet            500540
enviserv             492
feodo                263
flubot             29999
fobber_v1            298
fobber_v2            299
gameover           12000
gspy                 100
kfos                 124
locky               1158
madmax                 1
matsnu               908
mirai                  1
murofet             8560
mydoom             10047
necro               2708
necurs              8190
ngioweb             5274
nymaim               479
omexo                 40
padcrypt             168
proslikefan          100
pykspa_v1          44675
pykspa_v2_fake       800
pykspa_v2_real       199
qadars              2000
ramnit        

## Load top Alexa Legitimate domain

In [ ]:
Alexa_legit_columns = ["domain_name"]
Alexa_legit_domains = pd.read_csv('/content/drive/My Drive/Colab Notebooks/top-1m.csv', names=Alexa_legit_columns)
Alexa_legit_domains["label"] = "legit"
Alexa_legit_domains.head()

,domain_name,label
1,google.com,legit
2,youtube.com,legit
3,tmall.com,legit
4,baidu.com,legit
5,qq.com,legit


In [ ]:
Alexa_legit_domains.shape

(613032, 2)

## Concatinate both legitmate & DGA domains

In [ ]:
Netlab_dga_data.rename(columns = {'dga_family':'label'}, inplace = True)
Netlab_dga_data["label"] = "dga"
Netlab_dga_data.head()

,domain_name,label
0,odqzesdvd.info,dga
1,mcrwnxdzu.info,dga
2,poxrscvfoy.biz,dga
3,dupzwi.info,dga
4,pgldibr.com,dga


In [ ]:
# Adjusting datasets sizes and asserting that
Netlab_dga_data = Netlab_dga_data.loc[:Netlab_dga_data.shape[0]-1, :]

assert Netlab_dga_data.shape[0] == Netlab_dga_data.shape[0]

KeyError: ignored

In [ ]:
# Concatenating both datasets
Domain_data = pd.concat([Netlab_dga_data, Alexa_legit_domains])

print("Whole Dataset has {} rows".format(Domain_data.shape[0]))
assert Domain_data.shape[0] == Netlab_dga_data.shape[0] * 2
Domain_data.head()

In [ ]:
# Concatenating both datasets
Domain_data = pd.concat([Netlab_dga_data, Alexa_legit_domains])

print("Whole Dataset has {} rows".format(Domain_data.shape[0]))
Domain_data.head()

Whole Dataset has 2101627 rows


,domain_name,label
0,odqzesdvd.info,dga
1,mcrwnxdzu.info,dga
2,poxrscvfoy.biz,dga
3,dupzwi.info,dga
4,pgldibr.com,dga


## Dealing with Null and duplicate value 

In [ ]:
Domain_data.isnull().sum()

domain_name    0
label          0
dtype: int64

In [ ]:
Domain_data.duplicated().sum()

24989

In [ ]:
new_output = Domain_data[Domain_data.duplicated('domain_name')]
print("Duplicated values",new_output)

Duplicated values              domain_name label
34634         pocjubt.ru   dga
34635     qgvhlpwrjjo.ru   dga
34637        fsonck.info   dga
34638       wsfliiyg.com   dga
34639   tlffbudictx.info   dga
...                  ...   ...
428556      purymuq.info   dga
428557      hahogaz.info   dga
428559      masosom.info   dga
428564      pupygop.info   dga
428568      puvytuv.info   dga

[24989 rows x 2 columns]


In [ ]:
Domain_data = Domain_data.drop_duplicates(subset=['domain_name'])

In [ ]:
Domain_data.duplicated().sum()

0

In [ ]:
Domain_data.shape

(2076638, 2)

## Feature engineering

In [ ]:
! pip install tldextract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 93 kB 1.2 MB/s 


In [ ]:
def entropy(domain_name):
    """ Function which computes the entropy of a given domain name based on it's chars """
    elements, length = Counter(domain_name), len(domain_name)    
    
    return -sum(element/length * math.log(element/length, 2) for element in elements.values())

In [ ]:
import tldextract
def get_domain_name(domain):
    """ Function which extracts domain name from subdomain name """
    res = tldextract.extract(domain)
    return res.domain if len(res.domain) > len(res.subdomain) or entropy(res.domain) > entropy(res.subdomain) else res.subdomain

Domain_data["domain"] = Domain_data["domain_name"].apply(lambda domain: get_domain_name(domain))

In [ ]:
def firstDigitIndex( s ):
    for i, c in enumerate(s):
        if c.isdigit():
            return i + 1
    return 0

In [ ]:
def H_entropy (x):
    # Calculate Shannon Entropy
    prob = [ float(x.count(c)) / len(x) for c in dict.fromkeys(list(x)) ] 
    H = - sum([ p * np.log2(p) for p in prob ]) 
    return H

In [ ]:
def vowel_consonant_ratio (x):
    # Calculate vowel to consonant ratio
    x = x.lower()
    vowels_pattern = re.compile('([aeiou])')
    consonants_pattern = re.compile('([b-df-hj-np-tv-z])')
    vowels = re.findall(vowels_pattern, x)
    consonants = re.findall(consonants_pattern, x)
    try:
        ratio = len(vowels) / len(consonants)
    except: # catch zero devision exception 
        ratio = 0  
    return ratio

In [ ]:
def vowel_character_ratio (x):
    # Calculate vowel ratio
    x = x.lower()
    vowels_pattern = re.compile('([aeiou])')
    vowels = re.findall(vowels_pattern, x)
    try:
        ratio = len(vowels) / len(x)
    except: # catch zero devision exception 
        ratio = 0  
    return ratio

In [ ]:
def consonant_character_ratio (x):
    # Calculate consonant  ratio
    x = x.lower()
    consonants_pattern = re.compile('([b-df-hj-np-tv-z])')
    consonants = re.findall(consonants_pattern, x)
    try:
        ratio = len(consonants) / len(x)
    except: # catch zero devision exception 
        ratio = 0  
    return ratio

In [ ]:
def get_domain_level_number(domain_name):
  level= domain_name.split('.')
  return len(level)

In [ ]:
import socket
def is_valid_host(hostname):
    try:
        addr = socket.gethostbyname(hostname)
    except Exception:
        return 0
    return 1

In [ ]:
import socket
socket.gethostbyname('google.com')

'172.217.1.206'

In [ ]:
is_valid_host('owqyvra.in')

0

In [ ]:
is_valid_host('google.com')

1

In [ ]:
import re
# Applying functions above
Domain_data["length"] = Domain_data["domain"].apply(lambda domain: len(domain))
Domain_data["entropy"] = Domain_data["domain"].apply(lambda domain: entropy(domain))
Domain_data["first_Digit_Index"] = Domain_data["domain_name"].apply(lambda domain: firstDigitIndex(domain))
Domain_data["H_entropy"] = Domain_data["domain_name"].apply(lambda domain: H_entropy(domain))
Domain_data["vowel_consonant_ratio"] = Domain_data["domain_name"].apply(lambda domain: vowel_consonant_ratio(domain))
Domain_data["vowel_character_ratio"] = Domain_data["domain_name"].apply(lambda domain: vowel_character_ratio(domain))
Domain_data["consonant_character_ratio"] = Domain_data["domain_name"].apply(lambda domain: consonant_character_ratio(domain))
Domain_data["level_no"] = Domain_data["domain_name"].apply(lambda domain: get_domain_level_number(domain))  
Domain_data.head()

,domain_name,label,domain,length,entropy,first_Digit_Index,H_entropy,vowel_consonant_ratio,vowel_character_ratio,consonant_character_ratio,level_no
0,odqzesdvd.info,dga,odqzesdvd,9,2.641604,0,3.324863,0.444444,0.285714,0.642857,2
1,mcrwnxdzu.info,dga,mcrwnxdzu,9,3.169925,0,3.664498,0.300000,0.214286,0.714286,2
2,poxrscvfoy.biz,dga,poxrscvfoy,10,3.121928,0,3.664498,0.300000,0.214286,0.714286,2
3,dupzwi.info,dga,dupzwi,6,2.584963,0,3.277613,0.666667,0.363636,0.545455,2
4,pgldibr.com,dga,pgldibr,7,2.807355,0,3.459432,0.250000,0.181818,0.727273,2


In [ ]:
#from keras.callbacks import *
#Domain_data["Validation"] = Domain_data["domain_name"].apply(lambda domain_name: is_valid_host(domain_name))
#Domain_data.head()

In [ ]:
# A choice of 3 to 5 grams seems okay
import sklearn.feature_extraction

split_condition = Domain_data["label"] == "legit"
legit = Domain_data[split_condition]
dga = Domain_data[~split_condition]

alexa_vc = sklearn.feature_extraction.text.CountVectorizer(analyzer="char", ngram_range=(3,5), min_df=0.00001, max_df=1.0)
counts_matrix = alexa_vc.fit_transform(legit["domain"])

counts_matrix

<613032x184077 sparse matrix of type '<class 'numpy.int64'>'
	with 11887482 stored elements in Compressed Sparse Row format>

In [ ]:
alexa_counts = np.log10(np.asarray(counts_matrix.sum(axis=0)).flatten())
ngrams_list = alexa_vc.get_feature_names()
print(ngrams_list[100:200])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['--it', '--itb', '--j', '--j1', '--j1a', '--k', '--l', '--l3', '--l3c', '--m', '--m1', '--mg', '--mgb', '--n', '--o', '--p', '--q', '--r', '--s', '--t', '--th', '--w', '--y', '--ym', '--ymc', '--z', '-0-', '-01', '-1-', '-10', '-100', '-101', '-108', '-11', '-116', '-12', '-123', '-125', '-125-', '-12c', '-12ca', '-12cl', '-13', '-14', '-15', '-16', '-164', '-165', '-168', '-168-', '-17', '-18', '-180', '-180-', '-19', '-1x', '-1x2', '-2-', '-20', '-200', '-2001', '-201', '-202', '-2020', '-2021', '-2022', '-204', '-21', '-21-', '-22', '-22c', '-23', '-24', '-24-', '-240', '-25', '-28', '-3-', '-31', '-34', '-36', '-36-', '-360', '-365', '-37', '-37-', '-38', '-38-', '-3d', '-4-', '-40', '-41', '-42', '-42c', '-44', '-448', '-448a', '-48', '-49', '-4k']


In [ ]:
sorted_ngrams = sorted(zip(ngrams_list, alexa_counts), key=lambda alexa_count: alexa_count[1], reverse=True)
print("Alexa NGrams {}".format(len(sorted_ngrams)))
for ngram, count in sorted_ngrams[:10]:
    print(ngram, count)

Alexa NGrams 184077
ing 4.3406621988556084
ion 4.188253327026504
ent 4.180813775754397
ine 4.176467484599134
the 4.155123393710713
ter 4.151951788707265
lin 4.123198075031999
and 4.120047194353041
ers 4.074048069062633
tor 4.0604334067027406


In [ ]:
Domain_data["alexa_grams"] = alexa_counts * alexa_vc.transform(Domain_data["domain"]).T
Domain_data[Domain_data["label"] == "legit"].head()  

,domain_name,label,domain,length,entropy,first_Digit_Index,H_entropy,vowel_consonant_ratio,vowel_character_ratio,consonant_character_ratio,level_no,alexa_grams
1,google.com,legit,google,6,1.918296,0,2.646439,0.800000,0.400000,0.500000,2,24.600344
2,youtube.com,legit,youtube,7,2.521641,0,3.095795,1.000000,0.454545,0.454545,2,31.696975
3,tmall.com,legit,tmall,5,1.921928,0,2.725481,0.333333,0.222222,0.666667,2,16.423631
4,baidu.com,legit,baidu,5,2.321928,0,3.169925,1.000000,0.444444,0.444444,2,12.225260
5,qq.com,legit,qq,2,-0.000000,0,2.251629,0.250000,0.166667,0.666667,2,0.000000


In [ ]:
!pip install textfeatures

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
words_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/words.txt", names=["word"],
                             encoding="utf-8", header=None, dtype={"word": np.str})
words_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


,word
0,2
1,1080
2,&c
3,10-point
4,10th


In [ ]:
def clean_words_df(word):
    return str(word).strip().lower()

def keep_alphanumeric(word):
    return str(word).isalpha()



words_df = words_df[words_df["word"].map(lambda word: str(word).isalpha())]
words_df = words_df.applymap(lambda word: str(word).strip().lower())

words_df = words_df.dropna()
words_df = words_df.drop_duplicates()

In [ ]:
dict_cv = sklearn.feature_extraction.text.CountVectorizer(analyzer="char", ngram_range=(3,5), min_df=0.00001, max_df=1.0)
words_counts_matrix = dict_cv.fit_transform(words_df["word"])

dict_counts = np.log(np.asarray(words_counts_matrix.sum(axis=0)).flatten())
words_ngrams_list = dict_cv.get_feature_names()

print(words_ngrams_list[100:200])

['abbed', 'abber', 'abbes', 'abbet', 'abbey', 'abbi', 'abbie', 'abbil', 'abbin', 'abbis', 'abbit', 'abbl', 'abble', 'abbli', 'abbo', 'abbot', 'abbr', 'abbre', 'abbro', 'abby', 'abc', 'abd', 'abda', 'abde', 'abdi', 'abdic', 'abdit', 'abdo', 'abdoc', 'abdom', 'abdop', 'abdos', 'abdu', 'abduc', 'abe', 'abea', 'abeat', 'abec', 'abece', 'abecu', 'abed', 'abee', 'abef', 'abei', 'abel', 'abela', 'abele', 'abeli', 'abell', 'abels', 'abem', 'abema', 'aben', 'abena', 'abend', 'aber', 'aberd', 'aberg', 'abern', 'aberr', 'abers', 'abes', 'abet', 'abete', 'abeth', 'abeti', 'abeto', 'abett', 'abey', 'abeya', 'abez', 'abf', 'abh', 'abha', 'abho', 'abhor', 'abi', 'abia', 'abial', 'abian', 'abiat', 'abib', 'abic', 'abica', 'abich', 'abici', 'abid', 'abida', 'abide', 'abidi', 'abie', 'abies', 'abiet', 'abif', 'abifi', 'abify', 'abig', 'abiga', 'abil', 'abila']


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
Domain_data["words_grams"] = dict_counts * dict_cv.transform(Domain_data["domain"]).T
Domain_data.head()

,domain_name,label,domain,length,entropy,first_Digit_Index,H_entropy,vowel_consonant_ratio,vowel_character_ratio,consonant_character_ratio,level_no,alexa_grams,words_grams
0,odqzesdvd.info,dga,odqzesdvd,9,2.641604,0,3.324863,0.444444,0.285714,0.642857,2,8.618689,10.200253
1,mcrwnxdzu.info,dga,mcrwnxdzu,9,3.169925,0,3.664498,0.300000,0.214286,0.714286,2,5.331953,4.442651
2,poxrscvfoy.biz,dga,poxrscvfoy,10,3.121928,0,3.664498,0.300000,0.214286,0.714286,2,9.758886,11.231941
3,dupzwi.info,dga,dupzwi,6,2.584963,0,3.277613,0.666667,0.363636,0.545455,2,3.861893,7.928046
4,pgldibr.com,dga,pgldibr,7,2.807355,0,3.459432,0.250000,0.181818,0.727273,2,12.146719,20.785039


In [ ]:
Domain_data[Domain_data["label"] == "legit"].head(10)

,domain_name,label,domain,length,entropy,first_Digit_Index,H_entropy,vowel_consonant_ratio,vowel_character_ratio,consonant_character_ratio,level_no,alexa_grams,words_grams
1,google.com,legit,google,6,1.918296,0,2.646439,0.800000,0.400000,0.500000,2,24.600344,34.232630
2,youtube.com,legit,youtube,7,2.521641,0,3.095795,1.000000,0.454545,0.454545,2,31.696975,42.128381
3,tmall.com,legit,tmall,5,1.921928,0,2.725481,0.333333,0.222222,0.666667,2,16.423631,30.074280
4,baidu.com,legit,baidu,5,2.321928,0,3.169925,1.000000,0.444444,0.444444,2,12.225260,20.700835
5,qq.com,legit,qq,2,-0.000000,0,2.251629,0.250000,0.166667,0.666667,2,0.000000,0.000000
6,sohu.com,legit,sohu,4,2.000000,0,2.750000,0.750000,0.375000,0.500000,2,4.320105,7.662938
7,taobao.com,legit,taobao,6,1.918296,0,2.646439,1.250000,0.500000,0.400000,2,16.794742,14.418120
8,facebook.com,legit,facebook,8,2.750000,0,3.022055,0.833333,0.416667,0.500000,2,39.643030,64.179832
9,360.cn,legit,360,3,1.584963,1,2.584963,0.000000,0.000000,0.333333,2,2.775246,0.000000
10,amazon.com,legit,amazon,6,2.251629,0,2.721928,0.800000,0.400000,0.500000,2,23.787824,41.067154


In [ ]:
Domain_data.to_csv(r'/content/drive/My Drive/Colab Notebooks/MLDOMAIN_BRY.csv')